####Setting Environment


In [ ]:
!pip install transformers datasets torch accelerate gradio
!pip install wandb evaluate scikit-learn pandas numpy matplotlib seaborn
!pip install huggingface_hub tokenizers

In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    AutoModelForCausalLM, TrainingArguments, Trainer,
    pipeline, DataCollatorWithPadding
)
from datasets import Dataset, load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
!pip install --upgrade datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12

####Getting data

In [ ]:
emotions_dataset = load_dataset("go_emotions")


README.md:   0%|          | 0.00/9.40k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/350k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/347k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

In [ ]:
print(emotions_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 43410
    })
    validation: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5426
    })
    test: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5427
    })
})


In [ ]:
emotion_labels = emotions_dataset["train"].features["labels"].feature.names
print(f"\nEmotion labels ({len(emotion_labels)}): {emotion_labels}")


Emotion labels (28): ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [ ]:
empathetic_dataset = load_dataset("empathetic_dialogues")

README.md:   0%|          | 0.00/7.15k [00:00<?, ?B/s]

empathetic_dialogues.py:   0%|          | 0.00/4.51k [00:00<?, ?B/s]

The repository for empathetic_dialogues contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/empathetic_dialogues.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/76673 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10943 [00:00<?, ? examples/s]

In [ ]:
print(empathetic_dataset['train'][0])

{'conv_id': 'hit:0_conv:1', 'utterance_idx': 1, 'context': 'sentimental', 'prompt': 'I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world.', 'speaker_idx': 1, 'utterance': 'I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world.', 'selfeval': '5|5|5_2|2|5', 'tags': ''}


###Emotion Classification Model

In [ ]:
def preprocess_emotions_data(dataset):


    def process_example(example):
        # Convert multi-label to single dominant emotion
        labels = example['labels']
        if len(labels) > 0:
            # Take the first label as dominant emotion
            dominant_emotion = labels[0]
        else:
            dominant_emotion = 27  # neutral emotion index

        return {
            'text': example['text'],
            'label': dominant_emotion,
            'emotion_name': emotion_labels[dominant_emotion]
        }

    processed = dataset.map(process_example)
    return processed


In [ ]:
train_emotions = preprocess_emotions_data(emotions_dataset['train'])
val_emotions = preprocess_emotions_data(emotions_dataset['validation'])
test_emotions = preprocess_emotions_data(emotions_dataset['test'])

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

In [ ]:
print("Processed emotion dataset:")
print(f"Train: {len(train_emotions)}")
print(f"Validation: {len(val_emotions)}")
print(f"Test: {len(test_emotions)}")

Processed emotion dataset:
Train: 43410
Validation: 5426
Test: 5427


In [ ]:
model_name = "distilbert-base-uncased"
emotion_tokenizer = AutoTokenizer.from_pretrained(model_name)
emotion_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(emotion_labels)
)

print(f"Emotion model loaded: {model_name}")
print(f"Number of emotion classes: {len(emotion_labels)}")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Emotion model loaded: distilbert-base-uncased
Number of emotion classes: 28


In [ ]:
def tokenize_emotions(examples):
    return emotion_tokenizer(
        examples['text'],
        truncation=True,
        padding=True,
        max_length=128
    )
train_emotions_tokenized = train_emotions.map(tokenize_emotions, batched=True)
val_emotions_tokenized = val_emotions.map(tokenize_emotions, batched=True)


train_emotions_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_emotions_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

In [ ]:
from transformers import EarlyStoppingCallback

In [ ]:
emotion_training_args = TrainingArguments(
    output_dir='./emotion_classifier',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)


In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': accuracy_score(labels, predictions)}


In [ ]:
emotion_trainer = Trainer(
    model=emotion_model,
    args=emotion_training_args,
    train_dataset=train_emotions_tokenized,
    eval_dataset=val_emotions_tokenized,
    tokenizer=emotion_tokenizer,
    data_collator=DataCollatorWithPadding(emotion_tokenizer),
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
emotion_trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shravikajain79 (shravikajain79-institute-of-engineering-technology-davv) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy
500,1.931400,1.903429,0.462219
1000,1.602000,1.566068,0.549945
1500,1.594400,1.495080,0.557685
2000,1.492600,1.451933,0.571508
2500,1.493300,1.445751,0.560081
3000,1.300100,1.398084,0.574088
3500,1.191100,1.421419,0.580170
4000,1.203700,1.399376,0.574456
4500,1.270000,1.391460,0.577036
5000,1.147300,1.385120,0.585883


TrainOutput(global_step=6500, training_loss=1.3773355043851412, metrics={'train_runtime': 1326.8932, 'train_samples_per_second': 98.147, 'train_steps_per_second': 6.136, 'total_flos': 2561944649810976.0, 'train_loss': 1.3773355043851412, 'epoch': 2.3949889462048635})

###Response Generation Model

In [ ]:
def preprocess_empathetic_data(dataset):
    """Preprocess empathetic dialogues for response generation"""

    conversations = []

    for example in dataset:
        context = example['context']
        prompt = example['prompt']
        emotion = example['emotion']

        # Create input-output pairs
        input_text = f"Emotion: {emotion} | Context: {context} | User: {prompt}"
        output_text = example['utterance']

        conversations.append({
            'input': input_text,
            'output': output_text,
            'emotion': emotion
        })

    return conversations

In [ ]:
def preprocess_empathetic_data(dataset):
    """Preprocess empathetic dialogues for response generation"""

    conversations = []

    for example in dataset:
        # Print keys to inspect example structure
        print(example.keys())
        context = example['context']
        prompt = example['prompt']
        # Check if 'emotion' key exists before accessing
        if 'emotion' in example:
            emotion = example['emotion']
        else:
            # Handle cases where emotion might be missing or named differently
            print("Warning: 'emotion' key not found in example.")
            emotion = "unknown" # Assign a default or handle appropriately

        # Create input-output pairs
        input_text = f"Emotion: {emotion} | Context: {context} | User: {prompt}"
        output_text = example['utterance']

        conversations.append({
            'input': input_text,
            'output': output_text,
            'emotion': emotion
        })

    return conversations

# Rest of the code remains the same
train_conversations = preprocess_empathetic_data(empathetic_dataset['train'])
val_conversations = preprocess_empathetic_data(empathetic_dataset['validation'])

print(f"Processed conversations - Train: {len(train_conversations)}, Val: {len(val_conversations)}")
print("\nExample conversation:")
print(f"Input: {train_conversations[0]['input']}")
print(f"Output: {train_conversations[0]['output']}")

Streaming output truncated to the last 5000 lines.
dict_keys(['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'])
dict_keys(['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'])
dict_keys(['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'])
dict_keys(['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'])
dict_keys(['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'])
dict_keys(['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'])
dict_keys(['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'])
dict_keys(['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'])
dict_keys(['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'u

In [ ]:
response_model_name = "t5-small"
response_tokenizer = AutoTokenizer.from_pretrained(response_model_name)
from transformers import AutoModelForSeq2SeqLM
response_model = AutoModelForSeq2SeqLM.from_pretrained(response_model_name)


if response_tokenizer.pad_token is None:
    response_tokenizer.pad_token = response_tokenizer.eos_token

print(f"Response model loaded: {response_model_name}")

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Response model loaded: t5-small


In [ ]:
def tokenize_conversations(conversations, tokenizer, max_length=256):
    """Tokenize conversation data for training"""

    inputs = []
    targets = []

    for conv in conversations:

        input_text = f"Generate empathetic response: {conv['input']}"
        target_text = conv['output']

        inputs.append(input_text)
        targets.append(target_text)


    model_inputs = tokenizer(
        inputs,
        max_length=max_length,
        truncation=True,
        padding=True,
        return_tensors="pt"
    )

    labels = tokenizer(
        targets,
        max_length=max_length,
        truncation=True,
        padding=True,
        return_tensors="pt"
    )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


print("Tokenizing conversation data...")
train_conv_tokenized = tokenize_conversations(train_conversations[:5000], response_tokenizer)  # Limit for faster training
val_conv_tokenized = tokenize_conversations(val_conversations[:1000], response_tokenizer)

Tokenizing conversation data...


###Complete Chatbot Pipeline

In [1]:

save_directory = "./emotion_classifier_saved"


emotion_model.save_pretrained(save_directory)


emotion_tokenizer.save_pretrained(save_directory)

print(f"Trained model and tokenizer saved to {save_directory}")

try:

    emotion_classification_pipeline_instance = pipeline(
        "text-classification",
        model=save_directory,  # Use the explicitly saved local path
        tokenizer=save_directory, # Use the correct local path
        device=0 if torch.cuda.is_available() else -1
    )
    print("Emotion classifier pipeline loaded successfully from saved directory.")

except Exception as e:
    print(f"Failed to load pipeline from {save_directory}: {e}")

    try:
        # Rename the variable in the fallback as well
        emotion_classification_pipeline_instance = pipeline(
            "text-classification",
            model="./emotion_classifier",  # Original directory from Trainer
            tokenizer="./emotion_classifier", # Original directory from Trainer
            device=0 if torch.cuda.is_available() else -1
        )
        print("Emotion classifier pipeline loaded successfully from original Trainer directory.")
    except Exception as e_fallback:
        print(f"Failed to load pipeline from original Trainer directory as well: {e_fallback}")
        print("Emotion classifier pipeline could not be loaded. Please check the './emotion_classifier' directory contents.")
        # Set the variable to None or a dummy object if loading fails
        emotion_classification_pipeline_instance = None


def detect_emotion(text):
    """Detect emotion in user input"""

    if emotion_classification_pipeline_instance is None: # Check if the pipeline instance was successfully created
         print("Error: emotion_classifier pipeline not loaded.")
         return "neutral", 0.5

    try:
        # Use the new variable name to call the pipeline instance
        result = emotion_classification_pipeline_instance(text)
        # Ensure the result format is as expected before accessing keys
        if result and isinstance(result, list) and len(result) > 0 and isinstance(result[0], dict):
            emotion = result[0].get('label', 'neutral') # Use .get() for safer access
            confidence = result[0].get('score', 0.5)
            return emotion, confidence
        else:
            print(f"Warning: Unexpected result format from classifier: {result}")
            return "neutral", 0.5

    except Exception as e:
        print(f"Error during emotion detection: {e}")

        return "neutral", 0.5

test_texts = [
    "I'm feeling really sad today",
    "This is amazing! I'm so happy!",
    "I'm worried about my exam tomorrow",
    "I feel lonely and isolated"
]

print("Testing emotion detection:")
print("-" * 50)
for text in test_texts:
    emotion, confidence = detect_emotion(text)
    print(f"Text: '{text}' -> Emotion: {emotion} (confidence: {confidence:.3f})")

NameError: name 'emotion_model' is not defined

In [ ]:
# Simple response generation (rule-based + template)
class EmpatheticResponseGenerator:
    def __init__(self):
        self.emotion_responses = {
            'sadness': [
                "I'm sorry you're feeling this way. It's completely normal to feel sad sometimes.",
                "I understand you're going through a difficult time. Would you like to talk about what's bothering you?",
                "Your feelings are valid. Remember that it's okay to not be okay sometimes."
            ],
            'joy': [
                "That's wonderful! I'm so happy to hear that you're feeling good.",
                "It's great to see you in such a positive mood! What's making you feel so happy?",
                "I love your positive energy! It's contagious."
            ],
            'anger': [
                "I can sense you're feeling frustrated. Take a deep breath and know that these feelings will pass.",
                "It's natural to feel angry sometimes. Would you like to talk about what's causing this frustration?",
                "I hear your anger. Let's work through this together. What's bothering you?"
            ],
            'fear': [
                "I understand you're feeling anxious or scared. You're not alone in this.",
                "It's brave of you to share your fears. What specifically is worrying you?",
                "Fear is a natural emotion. Let's talk through what's making you feel this way."
            ],
            'surprise': [
                "That sounds unexpected! How are you processing this surprise?",
                "Surprises can be overwhelming. How are you feeling about this unexpected turn?",
                "That must have caught you off guard. Do you want to talk about it?"
            ],
            'love': [
                "Love is such a beautiful emotion. I'm glad you're experiencing it.",
                "It's wonderful that you're feeling love. That's a precious feeling to cherish.",
                "Love brings so much warmth to life. Tell me more about what's making you feel this way."
            ],
            'neutral': [
                "I'm here to listen. How are you feeling today?",
                "Thank you for sharing with me. What's on your mind?",
                "I'm here to support you. What would you like to talk about?"
            ]
        }

    def generate_response(self, user_input, detected_emotion):
        """Generate empathetic response based on emotion"""

        # Map emotion labels to our categories
        emotion_mapping = {
            'LABEL_0': 'neutral', 'LABEL_1': 'joy', 'LABEL_2': 'sadness',
            'LABEL_3': 'anger', 'LABEL_4': 'fear', 'LABEL_5': 'surprise',
            'LABEL_6': 'love'
        }

        emotion_category = emotion_mapping.get(detected_emotion, 'neutral')

        # Get appropriate responses for the emotion
        responses = self.emotion_responses.get(emotion_category, self.emotion_responses['neutral'])


        import random
        base_response = random.choice(responses)

        # Add personalized touch
        if len(user_input.split()) > 10:
            follow_up = " I can see this is important to you. Would you like to explore this further?"
            base_response += follow_up

        return base_response


response_generator = EmpatheticResponseGenerator()

In [ ]:
class MentalHealthChatbot:
    def __init__(self):
        self.emotion_classifier = emotion_classifier
        self.response_generator = EmpatheticResponseGenerator()
        self.conversation_history = []

    def chat(self, user_input):
        """Main chat function"""


        emotion, confidence = detect_emotion(user_input)


        response = self.response_generator.generate_response(user_input, emotion)


        self.conversation_history.append({
            'user': user_input,
            'emotion': emotion,
            'confidence': confidence,
            'bot': response
        })

        return response, emotion, confidence

    def get_conversation_history(self):
        return self.conversation_history

    def clear_history(self):
        self.conversation_history = []


chatbot = MentalHealthChatbot()

# Test the chatbot
test_inputs = [
    "I've been feeling really depressed lately and nothing seems to help",
    "I'm so excited about my new job! I can't wait to start!",
    "I'm really angry about what happened at work today",
    "I'm scared about my presentation tomorrow",
    "Hello, how are you?"
]

print("Testing complete chatbot:")
print("="*50)

for user_input in test_inputs:
    response, emotion, confidence = chatbot.chat(user_input)
    print(f"User: {user_input}")
    print(f"Detected Emotion: {emotion} (confidence: {confidence:.3f})")
    print(f"Bot: {response}")
    print("-"*50)

Testing complete chatbot:
User: I've been feeling really depressed lately and nothing seems to help
Detected Emotion: LABEL_25 (confidence: 0.494)
Bot: I'm here to listen. How are you feeling today? I can see this is important to you. Would you like to explore this further?
--------------------------------------------------
User: I'm so excited about my new job! I can't wait to start!
Detected Emotion: LABEL_13 (confidence: 0.943)
Bot: I'm here to support you. What would you like to talk about? I can see this is important to you. Would you like to explore this further?
--------------------------------------------------
User: I'm really angry about what happened at work today
Detected Emotion: LABEL_2 (confidence: 0.831)
Bot: I understand you're going through a difficult time. Would you like to talk about what's bothering you?
--------------------------------------------------
User: I'm scared about my presentation tomorrow
Detected Emotion: LABEL_14 (confidence: 0.943)
Bot: I'm here to

###Gradio Interface

In [ ]:
import gradio as gr

def chatbot_interface(message, history):
    """Gradio interface function"""


    response, emotion, confidence = chatbot.chat(message)


    response_with_emotion = f"{response}\n\n*Detected emotion: {emotion} (confidence: {confidence:.2f})*"


    history.append([message, response_with_emotion])

    return "", history

def clear_conversation():
    """Clear conversation history"""
    chatbot.clear_history()
    return []

# Create Gradio interface
with gr.Blocks(title="Mental Health Support Chatbot", theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🧠 Mental Health Support Chatbot

        This AI chatbot is designed to provide empathetic responses and emotional support.
        It detects your emotions and responds with understanding and care.

        **Disclaimer**: This is a supportive tool and not a replacement for professional mental health care.
        """
    )

    chatbot_component = gr.Chatbot(
        label="Conversation",
        height=400,
        value=[],
        show_label=True
    )

    with gr.Row():
        msg = gr.Textbox(
            label="Your message",
            placeholder="Type your message here...",
            lines=2,
            scale=4
        )
        send_btn = gr.Button("Send", scale=1, variant="primary")

    with gr.Row():
        clear_btn = gr.Button("Clear Conversation", variant="secondary")

    gr.Markdown(
        """
        ### How it works:
        1. **Emotion Detection**: The bot analyzes your message to detect emotions
        2. **Empathetic Response**: Generates supportive responses based on detected emotions
        3. **Continuous Support**: Maintains context throughout the conversation

        ### Supported Emotions:
        Joy, Sadness, Anger, Fear, Surprise, Love, and more...
        """
    )

    # Event handlers
    msg.submit(chatbot_interface, [msg, chatbot_component], [msg, chatbot_component])
    send_btn.click(chatbot_interface, [msg, chatbot_component], [msg, chatbot_component])
    clear_btn.click(clear_conversation, outputs=[chatbot_component])

# Launch the app
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://256c22235c14744a40.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://256c22235c14744a40.gradio.live


###Model Evaluation and Improvement

In [ ]:
# Evaluate emotion classifier on test set
def evaluate_emotion_classifier():
    """Evaluate the emotion classification model"""

    print("Evaluating emotion classifier...")

    # Get predictions on test set
    test_predictions = []
    test_labels = []

    for i in range(min(1000, len(test_emotions))):  # Test on subset
        text = test_emotions[i]['text']
        true_label = test_emotions[i]['label']

        # Get prediction
        emotion, confidence = detect_emotion(text)

        # Convert emotion label to number (simplified)
        pred_label = emotion_labels.index(emotion) if emotion in emotion_labels else 0

        test_predictions.append(pred_label)
        test_labels.append(true_label)

    # Calculate accuracy
    accuracy = accuracy_score(test_labels, test_predictions)
    print(f"Emotion Classification Accuracy: {accuracy:.3f}")

    return accuracy

# Run evaluation
accuracy = evaluate_emotion_classifier()

Evaluating emotion classifier...
Emotion Classification Accuracy: 0.082


In [ ]:
# Save complete chatbot
import pickle
import os

def save_chatbot(chatbot, path="./mental_health_chatbot"):
    """Save the complete chatbot"""

    os.makedirs(path, exist_ok=True)

    # Save the chatbot object
    with open(f"{path}/chatbot.pkl", "wb") as f:
        pickle.dump(chatbot, f)

    print(f"Chatbot saved to {path}")

def load_chatbot(path="./mental_health_chatbot"):
    """Load the complete chatbot"""

    with open(f"{path}/chatbot.pkl", "rb") as f:
        chatbot = pickle.load(f)

    print(f"Chatbot loaded from {path}")
    return chatbot

# Save the trained chatbot
save_chatbot(chatbot)

Chatbot saved to ./mental_health_chatbot


### Deployment

In [ ]:
# Create app.py for Hugging Face Spaces deployment
app_py_content = '''
import gradio as gr
import torch
from transformers import pipeline
import pickle
import os

# Load the saved chatbot
def load_chatbot():
    # Your chatbot loading code here
    pass

# Initialize chatbot
chatbot = load_chatbot()

def chatbot_interface(message, history):
    response, emotion, confidence = chatbot.chat(message)
    response_with_emotion = f"{response}\\n\\n*Detected emotion: {emotion} (confidence: {confidence:.2f})*"
    history.append([message, response_with_emotion])
    return "", history

# Create Gradio interface (same as above)
with gr.Blocks(title="Mental Health Support Chatbot") as demo:
    # ... (same interface code)
    pass

if __name__ == "__main__":
    demo.launch()
'''

# Save app.py
with open("app.py", "w") as f:
    f.write(app_py_content)

print("app.py created for Hugging Face Spaces deployment")

app.py created for Hugging Face Spaces deployment


In [2]:

requirements = """
torch>=1.9.0
transformers>=4.20.0
datasets>=2.0.0
gradio>=3.0.0
scikit-learn>=1.0.0
pandas>=1.3.0
numpy>=1.21.0
"""

with open("requirements.txt", "w") as f:
    f.write(requirements)

print("requirements.txt created")

requirements.txt created
